In [66]:
import numpy as np
import pandas as pd
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import keras_tuner as kt

In [21]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<h1>Preprocessing</h1>

In [57]:
names = ['subject-id', 'activity-code', 'timestamp_phone_accel', 'x_phone_accel', 'y_phone_accel', 'z_phone_accel']

path = r'wisdm-dataset/raw/phone/accel' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_phone_accel = pd.concat(li, axis=0, ignore_index=True)

frame_phone_accel['z_phone_accel'] = frame_phone_accel['z_phone_accel'].str.rstrip(';').astype('float64')

In [52]:
names = ['subject-id', 'activity-code', 'timestamp_phone_gyro', 'x_phone_gyro', 'y_phone_gyro', 'z_phone_gyro']

path = r'wisdm-dataset/raw/phone/gyro' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_phone_gyro = pd.concat(li, axis=0, ignore_index=True).drop(['subject-id'], axis=1)
frame_phone_gyro['z_phone_gyro'] = frame_phone_gyro['z_phone_gyro'].str.rstrip(';').astype(float)

In [53]:
names = ['subject-id', 'activity-code', 'timestamp_watch_accel', 'x_watch_accel', 'y_watch_accel', 'z_watch_accel']

path = r'wisdm-dataset/raw/watch/accel' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_watch_accel = pd.concat(li, axis=0, ignore_index=True).drop(['subject-id'], axis=1)
frame_watch_accel['z_watch_accel'] = frame_watch_accel['z_watch_accel'].str.rstrip(';').astype(float)

In [54]:
names = ['subject-id', 'activity-code', 'timestamp_watch_gyro', 'x_watch_gyro', 'y_watch_gyro', 'z_watch_gyro']

path = r'wisdm-dataset/raw/watch/gyro' # use your path
all_files = glob.glob(path + "/*.txt")

li = []

for filename in all_files:
    df = pd.read_csv(filename, names=names)
    li.append(df)

frame_watch_gyro = pd.concat(li, axis=0, ignore_index=True).drop(['subject-id'], axis=1)
frame_watch_gyro['z_watch_gyro'] = frame_watch_gyro['z_watch_gyro'].str.rstrip(';').astype(float)

In [58]:
frame_phone_accel_copy = frame_phone_accel.copy()

,subject-id,activity-code,timestamp_phone_accel,x_phone_accel,y_phone_accel,z_phone_accel
0,1600,A,252207666810782,-0.364761,8.793503,1.055084
1,1600,A,252207717164786,-0.879730,9.768784,1.016998
2,1600,A,252207767518790,2.001495,11.109070,2.619156
3,1600,A,252207817872794,0.450623,12.651642,0.184555
4,1600,A,252207868226798,-2.164352,13.928436,-4.422485
...,...,...,...,...,...,...
4804398,1650,S,357734078421000,-8.234077,-4.275110,-1.439407
4804399,1650,S,357734098619000,-8.080796,-4.418811,-1.415457
4804400,1650,S,357734118715000,-7.994575,-4.457131,-1.441802
4804401,1650,S,357734138868000,-8.040081,-4.366121,-1.496887


In [63]:
frame_phone_accel_copy.sort_values('timestamp_phone_accel')

,subject-id,activity-code,timestamp_phone_accel,x_phone_accel,y_phone_accel,z_phone_accel
884281,1611,O,268102711042,4.081131,-9.344620,1.961945
884282,1611,O,268153065046,2.726700,-9.153198,1.470413
884283,1611,O,268203419049,1.788895,-8.773758,0.297684
884284,1611,O,268253773053,2.394150,-9.277420,0.654739
884285,1611,O,268304127057,4.658661,-9.536713,1.672699
...,...,...,...,...,...,...
3854484,1644,B,1819802730955000,0.505349,-10.226734,-2.680027
3854485,1644,B,1819802751154000,1.302891,-9.714200,-1.204695
3854486,1644,B,1819802771250000,3.573369,-9.366921,-0.656235
3854487,1644,B,1819802791466000,5.197193,-7.285650,-0.431104


In [61]:
frame_phone_gyro

,activity-code,timestamp_phone_gyro,x_phone_gyro,y_phone_gyro,z_phone_gyro
0,A,252207918580802,-0.853210,0.297226,0.890182
1,A,252207968934806,-0.875137,0.015472,0.162231
2,A,252208019288809,-0.720169,0.388489,-0.284012
3,A,252208069642813,-0.571640,1.227402,-0.241669
4,A,252208119996817,-0.380493,1.202835,-0.213135
...,...,...,...,...,...
3608630,S,357733977635000,0.438889,0.171508,0.169377
3608631,S,357734017925000,0.444215,0.158724,0.197074
3608632,S,357734058325000,0.519849,0.037284,0.196009
3608633,S,357734098619000,0.545415,-0.086286,0.178964


In [38]:
X_train_full_phone_accel, X_test_phone_accel, y_train_full_phone_accel, y_test_phone_accel = train_test_split(frame_phone_accel[['timestamp_phone_accel', 'x_phone_accel', 'y_phone_accel', 'z_phone_accel']], frame_phone_accel[['activity-code']], test_size=0.1, random_state=150, stratify=frame_phone_accel['activity-code'])
X_train_phone_accel, X_valid_phone_accel, y_train_phone_accel, y_valid_phone_accel = train_test_split(X_train_full_phone_accel, y_train_full_phone_accel, test_size=X_test_phone_accel.shape[0], random_state=150)

X_train_full_phone_gyro, X_test_phone_gyro, y_train_full_phone_gyro, y_test_phone_gyro = train_test_split(frame_phone_gyro[['timestamp_phone_gyro', 'x_phone_gyro', 'y_phone_gyro', 'z_phone_gyro']], frame_phone_gyro[['activity-code']], test_size=0.1, random_state=150, stratify=frame_phone_gyro['activity-code'])
X_train_phone_gyro, X_valid_phone_gyro, y_train_phone_gyro, y_valid_phone_gyro = train_test_split(X_train_full_phone_gyro, y_train_full_phone_gyro, test_size=X_test_phone_gyro.shape[0], random_state=150)

X_train_full_watch_accel, X_test_watch_accel, y_train_full_watch_accel, y_test_watch_accel = train_test_split(frame_watch_accel[['timestamp_watch_accel', 'x_watch_accel', 'y_watch_accel', 'z_watch_accel']], frame_watch_accel[['activity-code']], test_size=0.1, random_state=150, stratify=frame_watch_accel['activity-code'])
X_train_watch_accel, X_valid_watch_accel, y_train_watch_accel, y_valid_watch_accel = train_test_split(X_train_full_watch_accel, y_train_full_watch_accel, test_size=X_test_watch_accel.shape[0], random_state=150)

X_train_full_watch_gyro, X_test_watch_gyro, y_train_full_watch_gyro, y_test_watch_gyro = train_test_split(frame_watch_gyro[['timestamp_watch_gyro', 'x_watch_gyro', 'y_watch_gyro', 'z_watch_gyro']], frame_watch_gyro[['activity-code']], test_size=0.1, random_state=150, stratify=frame_watch_gyro['activity-code'])
X_train_watch_gyro, X_valid_watch_gyro, y_train_watch_gyro, y_valid_watch_gyro = train_test_split(X_train_full_watch_gyro, y_train_full_watch_gyro, test_size=X_test_watch_gyro.shape[0], random_state=150)

<h3>Feature scaling and encoding</h3>

In [17]:
scaler = MinMaxScaler().set_output(transform='pandas')
scaled_X_train_phone_accel = scaler.fit_transform(X_train_phone_accel) 
scaled_X_valid_phone_accel = scaler.transform(X_valid_phone_accel)
scaled_X_test_phone_accel = scaler.transform(X_test_phone_accel)

In [18]:
encoder = OneHotEncoder(sparse=False).set_output(transform='pandas')
y_train_phone_accel_cat = encoder.fit_transform(y_train_phone_accel)
y_valid_phone_accel_cat = encoder.transform(y_valid_phone_accel)
y_test_phone_accel_cat = encoder.transform(y_test_phone_accel)

c:\Users\Shaun\miniconda3\envs\WISDM\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


<h1>Hyperparameter Tuning</h1>

In [10]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=1, max_value=5)
    n_neurons = hp.Int("n_neurons", min_value=10, max_value=100)
    learning_rate = hp.Float("learning_rate", min_value=1e-6, max_value=10, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "rmsprop", "adam", "adamax", "nadam", "adamw"])

    if optimizer == "sgd":
        momentum = hp.Float('momentum', min_value=0, max_value=1)
        nesterov = hp.Boolean('nesterov')
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=nesterov)

    elif optimizer == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

    elif optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    elif optimizer == "adamax":
        optimizer = tf.keras.optimizers.Adamax(learning_rate=learning_rate)

    elif optimizer == "nadam":
        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    else:
        optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)


    phone_accel_inp = tf.keras.Input(shape=(4,))
    phone_gyro_inp = tf.keras.Input(shape=(4,))
    watch_accel_inp = tf.keras.Input(shape=(4,))
    watch_gyro_inp = tf.keras.Input(shape=(4,))

    # Create dictionary to store hidden layers per input
    phone_accel_hidden_layer_dict = {}
    phone_gyro_hidden_layer_dict = {}
    watch_accel_hidden_layer_dict = {}
    watch_gyro_hidden_layer_dict = {}

    phone_accel_hidden_layer_dict['hidden1'] = tf.keras.layers.Dense(n_neurons, activation='relu')(phone_accel_inp)
    phone_gyro_hidden_layer_dict['hidden1'] = tf.keras.layers.Dense(n_neurons, activation='relu')(phone_gyro_inp)
    watch_accel_hidden_layer_dict['hidden1'] = tf.keras.layers.Dense(n_neurons, activation='relu')(watch_accel_inp)
    watch_gyro_hidden_layer_dict['hidden1'] = tf.keras.layers.Dense(n_neurons, activation='relu')(watch_gyro_inp)

    # Adds remaining n - 1 hidden layers
    for i in range(2, n_hidden + 1):
        phone_accel_hidden_layer_dict['hidden'+str(i)] = tf.keras.layers.Dense(n_neurons, activation='relu')(phone_accel_hidden_layer_dict['hidden'+str(i-1)])
        phone_gyro_hidden_layer_dict['hidden'+str(i)] = tf.keras.layers.Dense(n_neurons, activation='relu')(phone_gyro_hidden_layer_dict['hidden'+str(i-1)])
        watch_accel_hidden_layer_dict['hidden'+str(i)] = tf.keras.layers.Dense(n_neurons, activation='relu')(watch_accel_hidden_layer_dict['hidden'+str(i-1)])
        watch_gyro_hidden_layer_dict['hidden'+str(i)] = tf.keras.layers.Dense(n_neurons, activation='relu')(watch_gyro_hidden_layer_dict['hidden'+str(i-1)])
    
    # Concatenates outputs of last hidden layers per input
    concat = tf.keras.layers.Concatenate()([phone_accel_hidden_layer_dict['hidden'+str(n_hidden)],
                                            phone_gyro_hidden_layer_dict['hidden'+str(n_hidden)],
                                            watch_accel_hidden_layer_dict['hidden'+str(n_hidden)],
                                            watch_gyro_hidden_layer_dict['hidden'+str(n_hidden)]])

    out = tf.keras.layers.Dense(18, activation='softmax', name='Activity')(concat)

    model = tf.keras.Model([phone_accel_inp, phone_gyro_inp, watch_accel_inp, watch_gyro_inp], out)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
# To be able to tune parameters in the .fit() model
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)
    
    def fit(self, hp, model, X, y, **kwargs):
        batch_size = hp.Int('batch_size', min_value=2, max_value=32)
        return model.fit(X, y, batch_size=batch_size, **kwargs)


In [15]:
bayesian_opt_tuner = kt.BayesianOptimization(MyHyperModel(), objective='val_loss', seed=150, max_trials=100, overwrite=True, directory='wisdm', project_name='my_bayesian_search')
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [19]:
bayesian_opt_tuner.get_best_hyperparameters(num_trials=0)

[]

In [16]:
bayesian_opt_tuner.search(scaled_X_train_phone_accel.head(1000), y_train_phone_accel_cat.head(1000), validation_data=(scaled_X_valid_phone_accel.head(1000), y_valid_phone_accel_cat.head(1000)), epochs=500, callbacks=[early_stopping_cb])

NameError: name 'scaled_X_train_phone_accel' is not defined